In [1]:
# We are assuming the use of an American Call here, meaning that we can exercise early 
import numpy as np
class Option(object):
    def __init__(self, option_type=None, price=None, strike=None, sigma=None,
                 interest=None, expiry=None, steps=None, time_to_expiry = None):
        self.option_type = option_type
        self.price = price
        self.strike = strike
        self.sigma = sigma
        self.interest = interest
        self.expiry = expiry
        self.steps = steps
        self.time_to_expiry = time_to_expiry if time_to_expiry else self.steps - 1
    
    @property
    def dt(self):
        return np.divide(float(self.expiry), float(self.steps))
    
    @property
    def u(self):
        sqrt_dt = np.sqrt(self.dt)
        sigma_dt = np.multiply(self.sigma, sqrt_dt)
        return np.exp(sigma_dt)
    
    @property
    def d(self):
        sqrt_dt = np.sqrt(self.dt)
        sigma_dt = -np.multiply(self.sigma, sqrt_dt)
        return np.exp(sigma_dt)
    
    @property
    def p(self):
        r_dt = np.multiply(self.interest, self.dt)
        p = np.divide(float(np.exp(r_dt) - self.d), float(self.u - self.d))
        return p
    
    @property
    def discount_factor(self):
        return np.exp(np.multiply(-self.interest, 1/self.steps))
    
   
    # We can make a dict representing the tree that exists for the binomial model
    def layer_price(self):
        tree_size = self.steps
        t_0_price = self.price
        heap_tree = []
        layer_dict = {}
        for i in range(0, tree_size):
            layer_dict[i] = self.layer(i)
        return layer_dict
    
    # This is the representation of a layer function
    def layer(self, layer_number):
        number_of_nodes = layer_number+1
        layer_array = []
        for i in range(0, number_of_nodes):
            d_exponent = i
            u_exponent = layer_number - d_exponent
            abs_diff =np.absolute(u_exponent-d_exponent)
            if u_exponent > d_exponent:
                layer_array.append(np.multiply(self.price, np.power(self.u, abs_diff)))
            elif d_exponent > u_exponent:
                layer_array.append(np.multiply(self.price, np.power(self.d, abs_diff)))
            elif u_exponent == d_exponent:
                layer_array.append(self.price)
        return layer_array
    
    def calculate_current_options_price(self, previous_option_values):
        current_options_prices = []
        for i in range(0, len(previous_option_values)-1):
            p_up = previous_option_values[i]
            p_down = previous_option_values[i+1]
            expected_value = self.p * p_up + (1-self.p) * p_down
            current_options_prices.append(np.multiply(self.discount_factor, expected_value))
        return np.array(current_options_prices)
            
    def options_pricing(self, verbose):
        tree_size = self.steps
        options_dict = {}
        layer_dict = self.layer_price()
        for i in range(tree_size-1, -1, -1):
            layer_prices = layer_dict[i]
            strike_full = np.full(len(layer_prices), self.strike) # Array of strike prices
            if self.option_type == 'Call':
                strike_diff = np.subtract(layer_prices, strike_full)
            elif self.option_type == 'Put':
                strike_diff = np.subtract(strike_full, layer_prices)
            if i == self.time_to_expiry:  # Time of Expiry
                options_dict[i] = np.maximum(strike_diff, 0)
            elif i < self.time_to_expiry:
                previous_option_values = options_dict[i+1]
                option_present_value = self.calculate_current_options_price(previous_option_values)
                options_dict[i] = np.maximum(strike_diff, option_present_value)
            if i == self.time_to_expiry or i < self.time_to_expiry:
                if verbose:
                     print("Option chain at {}: {}". format(i, options_dict[i]))
        return options_dict
    
    def option_price_at_zero(self, verbose=True):
        print("Building Trees, ")
        return self.options_pricing(verbose)[0][0]
            

In [2]:
# The options tree listed in the example attached
option_c = Option(option_type='Call', price=10, strike=10, sigma=0.3, interest=0.03, expiry=1, steps=12, time_to_expiry=6)
print(option_c.option_price_at_zero())

Building Trees, 
Option chain at 6: [6.81380601 4.13982458 1.89109944 0.         0.         0.
 0.        ]
Option chain at 5: [5.44392683 2.99177486 0.92960056 0.         0.         0.        ]
Option chain at 4: [4.18969979 1.94097464 0.45696021 0.         0.        ]
Option chain at 3: [3.04152554 1.18531057 0.2246262  0.        ]
Option chain at 2: [2.09480353 0.6963053  0.11041864]
Option chain at 1: [1.38202117 0.39814502]
Option chain at 0: [0.88079116]
0.8807911581195348


In [3]:
# The actual option chain
# Please be patient, this could take about a minute to run
option_a = Option(option_type='Call', price=100, strike=120, sigma=0.3, interest=0.03, expiry=1, steps=1000)

print(option_a.option_price_at_zero(verbose=False))

Building Trees, 
6.282542860997076
